In [1]:
import dataclasses

import jax

from openpi.models import model as _model
from openpi.policies import droid_policy
from openpi.policies import policy_config as _policy_config
from openpi.shared import download
from openpi.training import config as _config
from openpi.training import data_loader as _data_loader

/home/t-qimhuang/miniconda3/envs/openpi/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Policy inference

The following example shows how to create a policy from a checkpoint and run inference on a dummy example.

In [2]:
# config = _config.get_config("pi0_fast_droid")
# checkpoint_dir = download.maybe_download("gs://openpi-assets/checkpoints/pi0_fast_droid")

from pathlib import Path
config = _config.get_config("pi0_fast_droid")
checkpoint_dir = Path("/home/t-qimhuang/code/openpi/checkpoints//pi0_fast_droid")

# from pathlib import Path
# config = _config.get_config("pi05_libero")
# checkpoint_dir = Path("/home/t-qimhuang/code/openpi/checkpoints/pi05_libero")

# Create a trained policy.
policy = _policy_config.create_trained_policy(config, checkpoint_dir)

# Run inference on a dummy example. This example corresponds to observations produced by the DROID runtime.
example = droid_policy.make_droid_example()
print("Example keys:", list(example.keys()))

print("Example shapes:", {k: type(v) for k, v in example.items()})

result = policy.infer(example)

print("Result keys:", list(result.keys()))
print("Result shapes:", {k: type(v) for k, v in result.items()})

# Delete the policy to free up memory.
del policy

print("Actions shape:", result["actions"].shape)

Example keys: ['observation/exterior_image_1_left', 'observation/wrist_image_left', 'observation/joint_position', 'observation/gripper_position', 'prompt']
Example shapes: {'observation/exterior_image_1_left': <class 'numpy.ndarray'>, 'observation/wrist_image_left': <class 'numpy.ndarray'>, 'observation/joint_position': <class 'numpy.ndarray'>, 'observation/gripper_position': <class 'numpy.ndarray'>, 'prompt': <class 'str'>}
Result keys: ['actions', 'policy_timing']
Result shapes: {'actions': <class 'numpy.ndarray'>, 'policy_timing': <class 'dict'>}
Actions shape: (10, 8)


# Working with a live model


The following example shows how to create a live model from a checkpoint and compute training loss. First, we are going to demonstrate how to do it with fake data.


In [3]:
config = _config.get_config("pi0_aloha_sim")

checkpoint_dir = download.maybe_download("gs://openpi-assets/checkpoints/pi0_aloha_sim")
key = jax.random.key(0)

# Create a model from the checkpoint.
model = config.model.load(_model.restore_params(checkpoint_dir / "params"))

# We can create fake observations and actions to test the model.
obs, act = config.model.fake_obs(), config.model.fake_act()

# Sample actions from the model.
loss = model.compute_loss(key, obs, act)
print("Loss shape:", loss.shape)

Loss shape: (1, 50)


Now, we are going to create a data loader and use a real batch of training data to compute the loss.

In [4]:
# Reduce the batch size to reduce memory usage.
config = dataclasses.replace(config, batch_size=2)

# Load a single batch of data. This is the same data that will be used during training.
# NOTE: In order to make this example self-contained, we are skipping the normalization step
# since it requires the normalization statistics to be generated using `compute_norm_stats`.
loader = _data_loader.create_data_loader(config, num_batches=1, skip_norm_stats=True)
obs, act = next(iter(loader))

# Sample actions from the model.
loss = model.compute_loss(key, obs, act)

# Delete the model to free up memory.
del model

print("Loss shape:", loss.shape)

TypeError: stack(): argument 'tensors' (position 1) must be tuple of Tensors, not Column